# Game Recommendation System
This notebook demonstrates how to build a recommendation system using content-based filtering. The system takes into account user preferences and game attributes to recommend games.

In [2]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

## Sample Game Data
We'll start by creating some sample game data.

In [3]:
# Sample data
game_data = {
    'appid': [2409360, 113020],
    'name': ["Pixel Cafe", "Monaco"],
    'playtime_forever': [300, 266],
    'genres': [["Casual", "Indie"], ["Strategy", "Indie"]],
    'platforms': [["windows"], ["windows", "mac"]],
    'price': [12.79, 15.99],
    'dlc_count': [2, 5],
    'recommendations': [78, 100],
    'categories': [["Single-player", "Steam Achievements"], ["Multiplayer", "Co-op"]]
}

# Convert to DataFrame
df = pd.DataFrame(game_data)
df

,appid,name,playtime_forever,genres,platforms,price,dlc_count,recommendations,categories
0,2409360,Pixel Cafe,300,"[Casual, Indie]",[windows],12.79,2,78,"[Single-player, Steam Achievements]"
1,113020,Monaco,266,"[Strategy, Indie]","[windows, mac]",15.99,5,100,"[Multiplayer, Co-op]"


## Normalizing Numerical Features
We'll normalize the numerical features using MinMaxScaler.

In [4]:
# Normalize numerical features
scaler = MinMaxScaler()
df[['playtime_forever', 'price', 'dlc_count', 'recommendations']] = scaler.fit_transform(df[['playtime_forever', 'price', 'dlc_count', 'recommendations']])
df

,appid,name,playtime_forever,genres,platforms,price,dlc_count,recommendations,categories
0,2409360,Pixel Cafe,1.0,"[Casual, Indie]",[windows],0.0,0.0,0.0,"[Single-player, Steam Achievements]"
1,113020,Monaco,0.0,"[Strategy, Indie]","[windows, mac]",1.0,1.0,1.0,"[Multiplayer, Co-op]"


## One-Hot Encoding
We need to convert categorical data (genres and platforms) into numerical data using one-hot encoding.

In [5]:
# One-hot encoding for genres and platforms
df = df.explode('genres')
df = pd.concat([df, pd.get_dummies(df['genres'])], axis=1).groupby(['appid', 'name']).sum().reset_index()

df = df.explode('platforms')
df = pd.concat([df, pd.get_dummies(df['platforms'], prefix='platform')], axis=1).groupby(['appid', 'name']).sum().reset_index()

df

,appid,name,playtime_forever,genres,platforms,price,dlc_count,recommendations,categories,Casual,Indie,Strategy,platform_mac,platform_windows
0,113020,Monaco,0.0,StrategyIndieStrategyIndieStrategyIndieStrateg...,windowsmacwindowsmac,8.0,8.0,8.0,"[Multiplayer, Co-op, Multiplayer, Co-op, Multi...",0,4,4,2,2
1,2409360,Pixel Cafe,4.0,CasualIndieCasualIndie,windowswindows,0.0,0.0,0.0,"[Single-player, Steam Achievements, Single-pla...",2,2,0,0,2


## User Preferences
Define user preferences to specify the importance of each attribute.

In [6]:
# Define user importance for each attribute
user_importance = {
    'playtime_forever': 0.1,
    'price': 0.2,
    'dlc_count': 0.1,
    'recommendations': 0.2,
    'Casual': 0.1,
    'Indie': 0.1,
    'Strategy': 0.1,
    'platform_windows': 0.1,
    'platform_mac': 0.1
}

# Display user preferences
user_importance

{'playtime_forever': 0.1,
 'price': 0.2,
 'dlc_count': 0.1,
 'recommendations': 0.2,
 'Casual': 0.1,
 'Indie': 0.1,
 'Strategy': 0.1,
 'platform_windows': 0.1,
 'platform_mac': 0.1}

## Weighted Scoring System
Calculate a weighted score for each game based on user preferences.

In [7]:
# Calculate weighted score for each game
df['score'] = df.apply(lambda row: sum([row[attr] * user_importance.get(attr, 0) for attr in user_importance]), axis=1)

# Sort games by score
recommended_games = df.sort_values(by='score', ascending=False)
recommended_games[['appid', 'name', 'score']]

,appid,name,score
0,113020,Monaco,5.2
1,2409360,Pixel Cafe,1.0


## Recommendation Function
Create a function that takes user preferences and game data, calculates scores, and returns recommendations.

In [8]:
def recommend_games(user_preferences, game_data):
    df = pd.DataFrame(game_data)
    
    # Normalize and preprocess data
    scaler = MinMaxScaler()
    df[['playtime_forever', 'price', 'dlc_count', 'recommendations']] = scaler.fit_transform(df[['playtime_forever', 'price', 'dlc_count', 'recommendations']])
    
    df = df.explode('genres')
    df = pd.concat([df, pd.get_dummies(df['genres'])], axis=1).groupby(['appid', 'name']).sum().reset_index()
    
    df = df.explode('platforms')
    df = pd.concat([df, pd.get_dummies(df['platforms'], prefix='platform')], axis=1).groupby(['appid', 'name']).sum().reset_index()
    
    # Calculate weighted score
    df['score'] = df.apply(lambda row: sum([row[attr] * user_preferences.get(attr, 0) for attr in user_preferences]), axis=1)
    
    # Sort and return recommended games
    recommended_games = df.sort_values(by='score', ascending=False)
    return recommended_games[['appid', 'name', 'score']]

## Using the Recommendation Function
Provide sample game data and user preferences to get game recommendations.

In [11]:
# Sample game data
game_data = {
    'appid': [2409360, 113020],
    'name': ["Pixel Cafe", "Monaco"],
    'playtime_forever': [719, 720],
    'genres': [["Casual", "Indie"], ["Strategy", "Indie"]],
    'platforms': [["windows"], ["windows", "mac"]],
    'price': [12.79, 15.99],
    'dlc_count': [6, 5],
    'recommendations': [99, 100],
    'categories': [["Single-player", "Steam Achievements"], ["Multiplayer", "Co-op"]]
}

# Define user preferences
user_preferences = {
    'playtime_forever': 0.1,
    'price': 0.2,
    'dlc_count': 0.1,
    'recommendations': 0.2,
    'Casual': 0.1,
    'Indie': 0.1,
    'Strategy': 0.1,
    'platform_windows': 0.1,
    'platform_mac': 0.1
}

# Recommend games
recommendations = recommend_games(user_preferences, game_data)
recommendations

,appid,name,score
0,113020,Monaco,5.2
1,2409360,Pixel Cafe,1.0
